# Mutations
In this notebook we will focus on each of the mutatons individually.

In [ ]:
%matplotlib inline
from scipy.stats import pearsonr
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import NMF, PCA
from sklearn.manifold import TSNE

from source import load_avenio_files
from transform import dummy_encode_mutations, get_top_correlated, mutation_train_test_split, patient_allele_frequencies


RANDOM_STATE = 1234
np.random.seed(RANDOM_STATE)

In [ ]:
# Load data from spreadsheet and SPSS files.
mutation_data_frame, phenotypes = load_avenio_files()

Two ways to calculate the differences:
- Using the allele frequencies.
- Using the mutatant concentration.

In [ ]:
# Vocabulary is the entire dataset, not only training set. Otherwise we run into problems during inference.
gene_vocabulary = mutation_data_frame['Gene'].unique()

# Columns containing allele frequencies.
mutant_allele_columns = ["T0: Allele \nFraction", "T1: Allele Fraction"]
# Columns containing mutant concentration.
mutant_concentration_columns = ["T0: No. Mutant \nMolecules per mL", "T1: No. Mutant \nMolecules per mL"]

# 1) Convert to float.
columns_to_numeric = mutant_allele_columns + mutation_concentration_columns
for column_name in columns_to_numeric:
    mutation_data_frame.loc[:, column_name] = pd.to_numeric(mutation_data_frame[column_name], errors='coerce')

# 2) Drop rows for which the columns can not be converted.
mutation_data_frame = mutation_data_frame.dropna(subset=columns_to_numeric)

Calculate the r-fraction as $$r = \frac{\Delta t}{t_0} \equiv \frac{t_1 - t_0}{t_0} .$$

In [ ]:
def r(t_0, t_1):
    return (t_1 - t_0) / t_0

In [ ]:
mutant_allele_frequencies = patient_allele_frequencies(
    mutation_data_frame, 
    gene_vocabulary, 
    # Calculate r(t_0, t_1).
    r,
    # Sum mutation values per gene in each patient.
    handle_duplicates="sum",
)

# Correlations
How do gene's fractions correlate? We calculate the Pearson correlation value which is defined as:
$$C_{ij} = \sum_{m=1}^{N} \frac{(X_{mi} - \mu_i)(X_{mj} - \mu_j)}{\sigma_i \sigma_j} \, ,$$
with $\mu$ and $\sigma$ the mean and standard deviation, respectively.

In [ ]:
def pearson_pval(x, y):
    return pearsonr(x, y)[1]

In [ ]:
corr = patient_mutation_frequencies.corr().fillna(0)
pval_corr = patient_mutation_frequencies.corr(method=pearson_pval).fillna(1)
corr.style.background_gradient(cmap='coolwarm', axis=None)

### Negative correlation

In [ ]:
gene_counts = mutation_data_frame['Gene'].value_counts()
get_top_correlated(
    corr, 
    pval_corr,
    gene_counts=gene_counts, 
    ascending=True, 
    top_count=4,
)

The p-values should not be taken to seriously. The fact that the p-values are extremely low is easy to understand:
- All columns are zero.
- Except the columns where the two unique mutations happen to coincide.
This immediately implies that the p-value should be near zero.

### Positive correlation

In [ ]:
get_top_correlated(corr, pval_corr, gene_counts=gene_counts, ascending=False)

## Do responders show an increase in mutational allel frequency?

Calculate quantities to comapre.

In [ ]:
mutation_data_frame['f_t'] = f_t(mutation_data_frame[allele_columns[0]], mutation_data_frame[allele_columns[1]])
mutation_data_frame['dt'] = mutation_data_frame[allele_columns[1]] - mutation_data_frame[allele_columns[0]]
mutation_data_frame['ratio'] = mutation_data_frame[allele_columns[1]] / mutation_data_frame[allele_columns[0]]

Add phenotype data.

In [ ]:
mutation_data_frame['response'] = mutation_data_frame['Patient ID'].apply(lambda x: phenotypes.loc[x, 'response_grouped'])
mutation_data_frame['progression'] = mutation_data_frame['Patient ID'].apply(lambda x: phenotypes.loc[x, 'progressie'])

In [ ]:
gene_subset = mutation_data_frame['Gene'].isin(['TP53', 'KRAS', 'PIK3CA', 'NFE2L2'])

g = sns.catplot(
    x='Gene', 
    y='f_t', 
    hue='response',
    data=mutation_data_frame[gene_subset],
    kind='violin',
)
g.fig.set_size_inches(16,8)

Absolute difference

In [ ]:
g = sns.catplot(
    x='Gene', 
    y='dt', 
    hue='response', 
#     col='progression', 
    data=mutation_data_frame[gene_subset],
#     col_wrap=1,
    kind='violin',
)
# g.ax.set_ylim([-.5, .5])
g.fig.set_size_inches(16,8)

In [ ]:
g = sns.catplot(
    x='Gene', 
    y='ratio', 
    hue='response', 
#     col='progression', 
    data=mutation_data_frame[gene_subset], 
    kind='violin',
)
g.fig.set_size_inches(16,8)
# g.ax.set_ylim([-.5, .5])